# Start of DS5559 Final Project

Team Left Twix Members

* Alice Wright - aew7j
* Edward Thompson - ejt8b
* Michael Davies -  mld9s
* Sam Parsons - sp8hp

In STAT 6021 members of our cohort looked at Transportation Network Company data sets to see if there was a potential relationship between tipping and other indicators, specifically with “transportation network providers” i.e. rideshares such as Uber, Lyft, etc.  At that point in our Data Science journey we did not have the skills or equipment to investigate this question in depth.  

Utilizing machine learning skills from SYS 6018 and applying Spark to this dataset we hope to come up with a more robust set of answers and potentially a better predictor of tipping. With other classification algorithms such as random forest and the heavy-weight data processing of Spark, will we be able to create a more robust predictive model?


Potential Questions from the TNC Data:

* Can it be predicted what fares are most likely to tip the driver?
* Is there a relationship between time of the fare and tipping? (workday stat, bar close, weekday, weekend, etc)
* Is there a relationship between start or end location of the ride and tipping? (downtown pickup, north shore, airport, etc)
* Is there a relationship between length or cost of ride and tipping? (do longer rides result in tips)
* Using this data would we be able to make recommendations to drivers to maximize likelihood of receiving a tip?
* Is the likelihood of tipping changing over time?  Are more rides being tipped?
* Are there re-identification abilities in this dataset? For instance, can we find records for a person who reliably takes a rideshare to/from work every day thereby linking a home address to a work address?




Additionally, joining in additional datasets may yield answers to questions about external factors such as:
* How did news reporting/social media on rideshare companies correlate with tipping?
* What relationship(s) does trip demand have with the stocks of these companies?

Data Source:
The best data source for this appears to be from the City of Chicago, as it is large (169M records and 21 columns), relatively clean, anonymized, and accessible via API.

City of Chicago:
https://data.cityofchicago.org/Transportation/Transportation-Network-Providers-Trips/m6dm-c72p

So far we have only pulled the data down via a CSV.

Code Rubric

* Data Import and PreProcessing | 2 pts

* Data splitting/sampling | 1 pt

* EDA (min two graphs) | 2 pts

* Model construction (min 3 models) | 3 pts

* Model evaluation | 2 pts

In [16]:
# import context manager: SparkSession
from pyspark.sql import SparkSession

# import data types
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, BooleanType
import pyspark.sql.types as typ
import pyspark.sql.functions as F
import os

from pyspark.mllib.evaluation import MulticlassMetrics

from pyspark.sql.types import *

spark = SparkSession.builder \
        .master("local[*]") \
        .appName("mllib_classifier") \
        .config("spark.executor.memory", '21g') \
        .config('spark.executor.cores', '2') \
        .config('spark.executor.instances', '3') \
        .config("spark.driver.memory",'1g') \
        .getOrCreate()
sc = spark.sparkContext

In [17]:
%whos

Variable                       Type               Data/Info
-----------------------------------------------------------
ArrayType                      type               <class 'pyspark.sql.types.ArrayType'>
Binarizer                      type               <class 'pyspark.ml.feature.Binarizer'>
BinaryType                     type               <class 'pyspark.sql.types.BinaryType'>
BooleanType                    type               <class 'pyspark.sql.types.BooleanType'>
ByteType                       type               <class 'pyspark.sql.types.ByteType'>
DataType                       type               <class 'pyspark.sql.types.DataType'>
DateType                       type               <class 'pyspark.sql.types.DateType'>
DecimalType                    type               <class 'pyspark.sql.types.DecimalType'>
DenseVector                    type               <class 'pyspark.ml.linalg.DenseVector'>
DoubleType                     type               <class 'pyspark.sql.types.DoubleT

In [ ]:
#clear old df
#del (df)

# Read in our Dataset

## Create a Custom Schema.  
This schema was been primarly determined by using a much smaller dataset and letting spark infer the schema.  We encountered an issue with spark reading in the ENTIRE dataset as NULL when there was a type mismatch.  Only the data we are likely to use later has been assigned to a specific type, otherwise it is left as a string type.

In [18]:
# create a custom schema.  

customSchema = StructType([
    StructField('Trip_ID', StringType(), True),        
    StructField('Trip_Start_Timestamp', StringType(), True),
    StructField('Trip_End_Timestamp', StringType(), True),
    StructField('Trip_Seconds', DoubleType(), True),
    StructField('Trip_Miles', DoubleType(), True),
    StructField('Pickup_Census_Tract', StringType(), True),
    StructField('Dropoff_Census_Tract', StringType(), True),
    StructField('Pickup_Community_Area', DoubleType(), True),
    StructField('Dropoff_Community_Area', DoubleType(), True),
    StructField("Fare", DoubleType(), True),
    StructField("Tip", DoubleType(), True),
    StructField("Additional_Charges", DoubleType(), True),
    StructField("Trip_Total", StringType(), True),
    StructField("Shared_Trip_Authorized", BooleanType(), True),
    StructField("Trips_Pooled", DoubleType(), True),
    StructField('Pickup_Centroid_Latitude', StringType(), True),
    StructField('Pickup_Centroid_Longitude', StringType(), True),
    StructField('Pickup_Centroid_Location', StringType(), True),
    StructField('Dropoff_Centroid_Latitude', StringType(), True),
    StructField('Dropoff_Centroid_Longitude', StringType(), True),
    StructField('Dropoff_Centroid_Location', StringType(), True)
])

#old readin.  Infer is slow for large dataset
#df = spark.read.csv('/../../project/ds5559/Alice_Ed_Michael_Sam_project/BigTrips.csv', header = True, inferSchema=True)

#read in the data to a dataframe
df = spark.read.csv('/../../project/ds5559/Alice_Ed_Michael_Sam_project/BigTrips.csv', header = True, schema=customSchema)
df.show(5)

+--------------------+--------------------+--------------------+------------+----------+-------------------+--------------------+---------------------+----------------------+----+---+------------------+----------+----------------------+------------+------------------------+-------------------------+------------------------+-------------------------+--------------------------+-------------------------+
|             Trip_ID|Trip_Start_Timestamp|  Trip_End_Timestamp|Trip_Seconds|Trip_Miles|Pickup_Census_Tract|Dropoff_Census_Tract|Pickup_Community_Area|Dropoff_Community_Area|Fare|Tip|Additional_Charges|Trip_Total|Shared_Trip_Authorized|Trips_Pooled|Pickup_Centroid_Latitude|Pickup_Centroid_Longitude|Pickup_Centroid_Location|Dropoff_Centroid_Latitude|Dropoff_Centroid_Longitude|Dropoff_Centroid_Location|
+--------------------+--------------------+--------------------+------------+----------+-------------------+--------------------+---------------------+----------------------+----+---+-------

In [19]:
rdd = sc.textFile('/../../project/ds5559/Alice_Ed_Michael_Sam_project/BigTrips.csv')
rdd = rdd.map(lambda x: x.split(","))

In [ ]:
rdd.take(1)

In [ ]:
#https://stackoverflow.com/questions/53304688/spark-date-format-mmm-dd-yyyy-hhmmss-am-to-timestamp-in-df
#https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html
start_times = rdd.map(lambda x: (x[0],x[1]))

In [ ]:
start_times = start_times.toDF()

In [ ]:
st = start_times.withColumn("_2",to_timestamp(col("_2"), "MM/dd/yyyy hh:mm:ss a"))

In [ ]:
st = start_times.withColumn("_2",to_timestamp(col("_2"), "MM/dd/yyyy hh:mm:ss a"))

In [20]:
#Doesn't update if you don't resave the variable

df = df.drop('Trip_End_Timestamp', 
             'Pickup_Census_Tract',
             'Dropoff_Census_Tract',
             'Pickup_Centroid_Latitude',
             'Pickup_Centroid_Longitude', 
             'Pickup_Centroid_Location', 
             'Dropoff_Centroid_Latitude', 
             'Dropoff_Centroid_Longitude', 
             'Dropoff_Centroid_Location')

In [21]:
df.printSchema()

root
 |-- Trip_ID: string (nullable = true)
 |-- Trip_Start_Timestamp: string (nullable = true)
 |-- Trip_Seconds: double (nullable = true)
 |-- Trip_Miles: double (nullable = true)
 |-- Pickup_Community_Area: double (nullable = true)
 |-- Dropoff_Community_Area: double (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Tip: double (nullable = true)
 |-- Additional_Charges: double (nullable = true)
 |-- Trip_Total: string (nullable = true)
 |-- Shared_Trip_Authorized: boolean (nullable = true)
 |-- Trips_Pooled: double (nullable = true)



In [6]:
df.count()

49108003

Make a sampled dataframe for faster work while developing all the steps.

In [22]:
df2 = df.sample(False, .005, seed = 2021).cache() #increased our sample size

In [23]:
df2.count()

244762

In [24]:
#delete the big df for now
del (df)

#hopefully that will make things faster 

In [ ]:
# df.select('Trip_ID').count()

In [ ]:
# df.describe()

From reading the data dictionary it appears that there are multiple ways that pickup and drop off locations are being reported.

Can we do histograms of the pickup and drop off areas?  To see if there are desinations that are popular (Airports, downtown, ball parks, museum row... etc)

## Fare and Tip Heatmap

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
plt.style.use('ggplot')

In [ ]:
#https://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html#module-pyspark.sql.functions
heatmap_data = df2.groupby('Fare','Tip').count().sort('Fare','Tip').groupby('Fare').pivot('Tip').sum("count").sort('Fare')
heatmap_data.cache()

In [ ]:
hm_data = np.array(heatmap_data.drop('Fare').toPandas())

In [ ]:
#https://numpy.org/doc/stable/reference/generated/numpy.flipud.html#numpy.flipud
hm_data = np.flipud(hm_data.transpose())

In [ ]:
#https://mungingdata.com/pyspark/column-to-list-collect-tolocaliterator/
fares = list(heatmap_data.select('Fare').toPandas()['Fare'])
tips = sorted([float(x) for x in heatmap_data.columns if x != 'Fare'], reverse=True)

In [ ]:
#https://www.pythonpool.com/matplotlib-figsize/
plt.figure(figsize=(20,15))
plt.axes().set_facecolor("#ffffff")
plt.axes().set_ylabel("Tip Amt")
plt.axes().set_xlabel("Fare Amt")
#https://www.pythonpool.com/matplotlib-heatmap/
plt.xticks(ticks=np.arange(len(fares)),labels=fares,rotation=90)
plt.yticks(ticks=np.arange(len(tips)),labels=tips)
hm = plt.imshow(hm_data,cmap='Blues',interpolation="none")
plt.colorbar(hm)
plt.title("Tip Amounts by Fare Amounts")

In [ ]:
heatmap_data = df2.groupby('Pickup_Community_Area','Dropoff_Community_Area').count().\
                    sort('Pickup_Community_Area','Dropoff_Community_Area').groupby('Pickup_Community_Area').pivot('Dropoff_Community_Area').sum("count").sort('Pickup_Community_Area')
#heatmap_data = heatmap_data.groupby('Pickup_Community_Area').pivot('Dropoff_Community_Area').count()
heatmap_data.cache()

In [ ]:
hm_data = np.array(heatmap_data.drop('Pickup_Community_Area').toPandas())

In [ ]:
hm_data = np.flipud(hm_data.transpose())
#https://mungingdata.com/pyspark/column-to-list-collect-tolocaliterator/
pickups = sorted(list(heatmap_data.select('Pickup_Community_Area').toPandas()['Pickup_Community_Area']))
dropoffs = sorted([float("nan") if x == "null" else float(x) for x in heatmap_data.columns if x != 'Pickup_Community_Area'], reverse=True)

In [ ]:
#https://www.pythonpool.com/matplotlib-figsize/
plt.figure(figsize=(20,20))
plt.axes().set_facecolor("#ffffff")
plt.axes().set_ylabel("Pickup Areas")
plt.axes().set_xlabel("Dropoff Areas")
#https://www.pythonpool.com/matplotlib-heatmap/
plt.xticks(ticks=np.arange(len(pickups)),labels=pickups,rotation=90)
plt.yticks(ticks=np.arange(len(dropoffs)),labels=dropoffs)
hm = plt.imshow(hm_data,cmap='Blues',interpolation="none")
plt.colorbar(hm)
plt.title("Tip Heatmap by Pickup and Dropoff Community Areas")

In [ ]:
heatmap_data = df2.groupby('Pickup_Community_Area','Dropoff_Community_Area').count().\
                    sort('Pickup_Community_Area','Dropoff_Community_Area').groupby('Pickup_Community_Area').pivot('Dropoff_Community_Area').sum("count").sort('Pickup_Community_Area')
#heatmap_data = heatmap_data.groupby('Pickup_Community_Area').pivot('Dropoff_Community_Area').count()
heatmap_data.cache()

In [ ]:
hm_data = np.array(heatmap_data.drop('Pickup_Community_Area').toPandas())

In [ ]:
hm_data = np.flipud(hm_data.transpose())
#https://mungingdata.com/pyspark/column-to-list-collect-tolocaliterator/
pickups = sorted(list(heatmap_data.select('Pickup_Community_Area').toPandas()['Pickup_Community_Area']))
dropoffs = sorted([float("nan") if x == "null" else float(x) for x in heatmap_data.columns if x != 'Pickup_Community_Area'], reverse=True)

In [ ]:
#https://www.pythonpool.com/matplotlib-figsize/
plt.figure(figsize=(20,20))
plt.axes().set_facecolor("#ffffff")
plt.axes().set_ylabel("Pickup Areas")
plt.axes().set_xlabel("Dropoff Areas")
#https://www.pythonpool.com/matplotlib-heatmap/
plt.xticks(ticks=np.arange(len(pickups)),labels=pickups,rotation=90)
plt.yticks(ticks=np.arange(len(dropoffs)),labels=dropoffs)
hm = plt.imshow(hm_data,cmap='Blues',interpolation="none")
plt.colorbar(hm)
plt.title("Tip Heatmap by Pickup and Dropoff Community Areas")

In [ ]:
pickups_hist = df2.groupby('Pickup_Community_Area').count().withColumnRenamed("count","area_count").sort("area_count")
#https://stackoverflow.com/questions/38610559/convert-spark-dataframe-column-to-python-list
areas = [str(x.Pickup_Community_Area) for x in pickups_hist.collect()]
area_count = [x.area_count for x in pickups_hist.collect()]
plt.figure(figsize=(35,10))
plt.bar(areas,area_count, color='lightblue')
plt.title('Pickup Community Areas Histogram')
plt.axes().set_facecolor("#ffffff")

In [ ]:
dropoffs_hist = df2.groupby('Dropoff_Community_Area').count().withColumnRenamed("count","area_count").sort("area_count")
areas = [str(x.Dropoff_Community_Area) for x in dropoffs_hist.collect()]
area_count = [x.area_count for x in dropoffs_hist.collect()]
plt.figure(figsize=(35,10))
plt.bar(areas,area_count, color='lightblue')
plt.title('Dropoff Community Areas Histogram')
plt.axes().set_facecolor("#ffffff")

Only showing top 10 as we have pretty large fall off.  Now to replace nulls with actual values.  Will use 99 for areas outside the city

In [ ]:
#now lets sort the list, added desc to order the list from largest to smallest

df2.groupby('Pickup_Community_Area').count().orderBy('count', ascending=False).show(10)

Just for reference https://www.google.com/maps/d/u/0/viewer?ll=41.91066822076546%2C-87.63676464181398&spn=0.340714%2C0.699692&msa=0&mid=1O-3Uot4mSetKW-M_govahruUjDc&z=12

Top Pickup Areas

* 8 is the area of Magnificiant Mile (high end shopping), the riverwalk (tourism, boat tours), Gold Coast Neigherborhood (very high income neigherhood), and Navy Pier (tourist trap)
* 28 is the Near West Side neigherborhoods of West Loop, Greektown, and Fulton's Market.  Lots of new condos and resturants.  Also has University of Illinois Chicago, the Medical District (University and VA hospitals) and the United Center (home of the Bull and Blackhawks)
* null is outside the city
* 32 is "The Loop" downtown busisness district, train El hub
* 6 is Lakeview, Near north neigherbood perdominatly white and has the Chciago Cubs Stadium
* 24 West Town and has the neigherborhoods of Wicker Park, Ukranian Village, and River West.  These neigherbohoods have all been ungergroing gentrification.
* 7 Lincoln Park.  Like Lakeview but more expensive.  Was one of the first neigherborhoods in Chicago to gentrify.  
* 22 Logan Square.  Edge of transitioning neigherbood.  More afforable for new gentrifiying owners.
* 76 O'Hare Airport
* 3 Uptown, north of Lakeview, similar to Logan Square.

In [ ]:
# do the same for the dropoffs
df2.groupby('Dropoff_Community_Area').count().orderBy('count', ascending=False).show(10)

In [ ]:
df2.groupby('Dropoff_Community_Area').count().orderBy('Dropoff_Community_Area', ascending=False).show()

just for reference https://www.google.com/maps/d/u/0/viewer?ll=41.91066822076546%2C-87.63676464181398&spn=0.340714%2C0.699692&msa=0&mid=1O-3Uot4mSetKW-M_govahruUjDc&z=12

Dropoff Areas

* 8is the area of Magnificiant Mile (high end shopping), the riverwalk (tourism, boat tours), Gold Coast Neigherborhood (very high income neigherhood), and Navy Pier (tourist trap)
* 32 is "The Loop" downtown busisness district, train El hub
* 28 is the Near West Side neigherborhoods of West Loop, Greektown, and Fulton's Market.  Lots of new condos and resturants.  Also has University of Illinois Chicago, the Medical District (University and VA hospitals) and the United Center (home of the Bull and Blackhawks)
* null is outside the city
* 6 is Lakeview, Near north neigherbood perdominatly white and has the Chciago Cubs Stadium
* 24 West Town and has the neigherborhoods of Wicker Park, Ukranian Village, and River West.  These neigherbohoods have all been ungergroing gentrification.
* 7 Lincoln Park. Like Lakeview but more expensive.  Was one of the first neigherborhoods in Chicago to gentrify.  
* 76 O'Hare Airport
* 22 Logan Square.  Edge of transitioning neigherbood.  More afforable for new gentrifiying owners.
* 3 Uptown, north of Lakeview, similar to Logan Square.

The lists are almost identical, just a few order changes.

How much of the traffic comes from these heavy use neigherborhoods?  

In [ ]:
# replace our null values in the pickup and dropoff locations
# https://stackoverflow.com/questions/42312042/how-to-replace-all-null-values-of-a-dataframe-in-pyspark

# choosing 78 as it is not a Chicago Community Area and using a larger number will result in a larger OHE matrix later.

In [25]:
df3 = df2.na.fill(value=78,subset=['Pickup_Community_Area', 'Dropoff_Community_Area'])

In [26]:
del(df2)

In [27]:
df3.groupby('Dropoff_Community_Area').count().orderBy('count', ascending=False).show(10)

+----------------------+-----+
|Dropoff_Community_Area|count|
+----------------------+-----+
|                   8.0|33047|
|                  78.0|19837|
|                  28.0|19285|
|                  32.0|18833|
|                   6.0|14692|
|                  24.0|12973|
|                   7.0|11656|
|                  76.0| 8499|
|                  22.0| 7867|
|                   3.0| 5058|
+----------------------+-----+
only showing top 10 rows



In [ ]:
df3.groupby('Pickup_Community_Area').count().orderBy('count', ascending=False).show(10)

looks like it works to change the nulls to a 78 for the community area.

Next lets add a colum with the tip or no tip as a binary

In [ ]:
df3.show(20)

In [ ]:
#old code, Michaels worked better.  This should be deleted later

#do we need this lit function?
#https://hackersandslackers.com/transforming-pyspark-dataframes/
#from pyspark.sql.functions import lit, when, col
#df4 = df3.withColumn('testColumn', F.lit('this is a test'))
# that worked
#df4 = df3.withColumn('Tip_Bool', when((col("tip") > 0), 1).otherwise(0))
# df = df.withColumn([COLUMN_NAME]. F.when([CONDITIONAL], [COLUMN_VALUE]).otherwsie([COLUMN_VALUE]))

In [28]:
# the biniazier that Michael found

# https://spark.apache.org/docs/2.2.0/ml-features.html#binarizer

from pyspark.ml.feature import Binarizer
#binarizer = Binarizer(threshold=0.5, inputCol="feature", outputCol="binarized_feature")
binarizer = Binarizer(threshold=0, inputCol="Tip", outputCol="binarized_tip")
binarizedTip = binarizer.transform(df3)
df4=binarizedTip.cache()

display(binarizedTip)

DataFrame[Trip_ID: string, Trip_Start_Timestamp: string, Trip_Seconds: double, Trip_Miles: double, Pickup_Community_Area: double, Dropoff_Community_Area: double, Fare: double, Tip: double, Additional_Charges: double, Trip_Total: string, Shared_Trip_Authorized: boolean, Trips_Pooled: double, binarized_tip: double]

In [29]:
del(df3)

In [30]:
df4.show(5)

+--------------------+--------------------+------------+----------+---------------------+----------------------+----+---+------------------+----------+----------------------+------------+-------------+
|             Trip_ID|Trip_Start_Timestamp|Trip_Seconds|Trip_Miles|Pickup_Community_Area|Dropoff_Community_Area|Fare|Tip|Additional_Charges|Trip_Total|Shared_Trip_Authorized|Trips_Pooled|binarized_tip|
+--------------------+--------------------+------------+----------+---------------------+----------------------+----+---+------------------+----------+----------------------+------------+-------------+
|1d7fc864aa5f775df...|12/01/2019 12:15:...|       543.0|       3.1|                  7.0|                   8.0| 7.5|0.0|              2.55|     10.05|                 false|         1.0|          0.0|
|1dc1dd5e6d3fb5bf1...|12/01/2019 12:15:...|      1242.0|      19.5|                 32.0|                  28.0| 5.0|0.0|               0.0|         5|                  true|         3.0|     

In [ ]:
df4.groupby('Dropoff_Community_Area', 'binarized_tip').count().orderBy('count', ascending=False).show(50)

i guess now we should start doing more statistics?

In [ ]:
df4.groupby("binarized_tip").count().show()

can we do a logistic model with what we have now?

In [31]:
# https://medium.com/@nutanbhogendrasharma/role-of-onehotencoder-and-pipelines-in-pyspark-ml-feature-part-2-3275767e74f0
# apparently the spark doc is wrong, we don't need fit, just transform.  Maybe thats a spark 3 thing?

from pyspark.ml.feature import OneHotEncoder

#onehotencoder to pickup
onehotencoder_pickup_vector = OneHotEncoder(inputCol="Pickup_Community_Area", outputCol="Pickup_Community_Area_vec")
df4 = onehotencoder_pickup_vector.transform(df4)

#onehotencoder to dropoff
onehotencoder_dropoff_vector = OneHotEncoder(inputCol="Dropoff_Community_Area", outputCol="Dropoff_Community_Area_vec")
df4 = onehotencoder_dropoff_vector.transform(df4).cache()

df4.show(20)

+--------------------+--------------------+------------+----------+---------------------+----------------------+----+---+------------------+----------+----------------------+------------+-------------+-------------------------+--------------------------+
|             Trip_ID|Trip_Start_Timestamp|Trip_Seconds|Trip_Miles|Pickup_Community_Area|Dropoff_Community_Area|Fare|Tip|Additional_Charges|Trip_Total|Shared_Trip_Authorized|Trips_Pooled|binarized_tip|Pickup_Community_Area_vec|Dropoff_Community_Area_vec|
+--------------------+--------------------+------------+----------+---------------------+----------------------+----+---+------------------+----------+----------------------+------------+-------------+-------------------------+--------------------------+
|1d7fc864aa5f775df...|12/01/2019 12:15:...|       543.0|       3.1|                  7.0|                   8.0| 7.5|0.0|              2.55|     10.05|                 false|         1.0|          0.0|           (78,[7],[1.0])|        

In [32]:
df4.cache()

DataFrame[Trip_ID: string, Trip_Start_Timestamp: string, Trip_Seconds: double, Trip_Miles: double, Pickup_Community_Area: double, Dropoff_Community_Area: double, Fare: double, Tip: double, Additional_Charges: double, Trip_Total: string, Shared_Trip_Authorized: boolean, Trips_Pooled: double, binarized_tip: double, Pickup_Community_Area_vec: vector, Dropoff_Community_Area_vec: vector]

In [33]:
df4.printSchema()

root
 |-- Trip_ID: string (nullable = true)
 |-- Trip_Start_Timestamp: string (nullable = true)
 |-- Trip_Seconds: double (nullable = true)
 |-- Trip_Miles: double (nullable = true)
 |-- Pickup_Community_Area: double (nullable = false)
 |-- Dropoff_Community_Area: double (nullable = false)
 |-- Fare: double (nullable = true)
 |-- Tip: double (nullable = true)
 |-- Additional_Charges: double (nullable = true)
 |-- Trip_Total: string (nullable = true)
 |-- Shared_Trip_Authorized: boolean (nullable = true)
 |-- Trips_Pooled: double (nullable = true)
 |-- binarized_tip: double (nullable = true)
 |-- Pickup_Community_Area_vec: vector (nullable = true)
 |-- Dropoff_Community_Area_vec: vector (nullable = true)



In [34]:
predictor_col_for_lr = ['Trip_Seconds',
                        'Trip_Miles',
                        'Fare',
                        'Additional_Charges',
                        'Shared_Trip_Authorized',
                        'Trips_Pooled',
                        'Pickup_Community_Area_vec',
                        'Dropoff_Community_Area_vec']

# removed to see if works better
#  'Pickup_Community_Area',
#                         'Dropoff_Community_Area',

# leaving out Trip_start_Timestamp for now as I don't know know to use the string

In [35]:
from pyspark.ml.linalg import DenseVector
from pyspark.ml.feature import VectorAssembler 
from pyspark.mllib.linalg import Vectors

In [36]:
# from last weeks homework
# use all of the fields as features

#not sure if we should be scaling the OHE vectors but we'll try first

assembler = VectorAssembler(inputCols=predictor_col_for_lr, outputCol="features") 
transformed = assembler.transform(df4)
df5 = transformed.cache()

df5.show(5)

+--------------------+--------------------+------------+----------+---------------------+----------------------+----+---+------------------+----------+----------------------+------------+-------------+-------------------------+--------------------------+--------------------+
|             Trip_ID|Trip_Start_Timestamp|Trip_Seconds|Trip_Miles|Pickup_Community_Area|Dropoff_Community_Area|Fare|Tip|Additional_Charges|Trip_Total|Shared_Trip_Authorized|Trips_Pooled|binarized_tip|Pickup_Community_Area_vec|Dropoff_Community_Area_vec|            features|
+--------------------+--------------------+------------+----------+---------------------+----------------------+----+---+------------------+----------+----------------------+------------+-------------+-------------------------+--------------------------+--------------------+
|1d7fc864aa5f775df...|12/01/2019 12:15:...|       543.0|       3.1|                  7.0|                   8.0| 7.5|0.0|              2.55|     10.05|                 fals

In [ ]:
# Feature scaling from our homework

#from https://spark.apache.org/docs/latest/ml-features#standardscaler

from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        withStd=True, withMean=False)

# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(df5)

# Normalize each feature to have unit standard deviation.
scaledData = scalerModel.transform(df5)


df6 = scaledData


In [ ]:
scaledData.show(5, False)

In [ ]:
# our model didn't work on the standard test train split.  Prof. Tashman recomended upscalling the help with the imbalanced dataset.
#from https://spark.apache.org/docs/2.1.0/ml-tuning.html#train-validation-split

train_inital, test = df6.randomSplit([0.8, 0.2], seed=2021)

train_inital.show(5)

test.cache()
test.show(5)

In [ ]:
# oversampleing code sample
# https://stackoverflow.com/questions/53273133/how-to-perform-up-sampling-using-sample-functionpy-spark

df_a = train_inital.filter(train_inital['binarized_tip'] == 0)
df_b = train_inital.filter(train_inital['binarized_tip'] == 1)


In [ ]:
a_count = df_a.count()
b_count = df_b.count() 
print(a_count)
print(b_count)


In [ ]:
ratio = a_count / b_count
print(ratio)

In [ ]:
df_b_overampled = df_b.sample(withReplacement=True, fraction=ratio, seed=1)

In [ ]:
train_final = df_a.unionAll(df_b_overampled)
train_final.cache()

In [ ]:
train_final.show(5)

In [ ]:
df_af = train_final.filter(train_inital['binarized_tip'] == 0)
df_bf = train_final.filter(train_inital['binarized_tip'] == 1)
a_count = df_af.count()
b_count = df_bf.count() 
print(a_count)
print(b_count)


Our training size has increased.  This is to be expected in upscaling.

Good reference: https://www.kaggle.com/rafjaa/resampling-strategies-for-imbalanced-datasets

In [ ]:
#just as a reminder what was the truth in our test data?

dft_a = test.filter(train_inital['binarized_tip'] == 0)
dft_b = test.filter(train_inital['binarized_tip'] == 1)
count_test_a = dft_a.count()
count_test_b = dft_b.count()
print(count_test_a)
print(count_test_b)


In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
# from the docs https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression

# Load training data
# training = spark.read.format("libsvm").load("data/mllib/sample_libsvm_data.txt")

#elastic net was killing all of our predictors... maybe that was good but I've turned it off for now.  Was at 0.8

lr = LogisticRegression(maxIter=3,
                        regParam=0.1,
                        elasticNetParam=0.3,
                        featuresCol="features",
                        labelCol="binarized_tip")

# Fit the model
lrModel = lr.fit(train_final)

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

# not doing multinominal... this was from the example
# We can also use the multinomial family for binary classification
# mlr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, family="multinomial")

# Fit the model
# mlrModel = mlr.fit(train)

# Print the coefficients and intercepts for logistic regression with multinomial family
# print("Multinomial coefficients: " + str(mlrModel.coefficientMatrix))
# print("Multinomial intercepts: " + str(mlrModel.interceptVector))

Make Predictions

In [ ]:
prediction = lrModel.transform(test)
prediction.show(5)

df7 = prediction

In [ ]:
df7.count()

classificiation evaluator

In [ ]:
# I can't find how to do this in a Dataframe.  Rip to an RDD I guess.  This feels stupid.

pred_rdd= df7.select('prediction').rdd.flatMap(lambda x: x)
label_rdd = df7.select('binarized_tip').rdd.flatMap(lambda x: x)

In [ ]:
pred_rdd.take(5)

In [ ]:
label_rdd.take(5)

In [ ]:
pred_rdd.count()

In [ ]:
label_rdd.count()

In [ ]:
#looks like we can zip them together

predictionAndLabels =  pred_rdd.zip(label_rdd)

In [ ]:
predictionAndLabels.take(5)

In [ ]:
metrics2 = MulticlassMetrics(predictionAndLabels)

cm1 = metrics2.confusionMatrix().toArray()
acc = (cm1[0][0] + cm1[1][1])/(cm1[0][0] + cm1[1][1] + cm1[0][1] + cm1[1][0]) #should be a function

In [ ]:
print(cm1)
print()
print(acc)

In [ ]:
#Tashman's sugestion of upsampleing worked.  We no longer have a useless classifier

#well that doesn't look aweseome.  Looks like we are only predicting no tip.  Might have to do some weighting magic.  Despite confusing myself, the number of results are correct since only 20% was used for the test, and 80% used in the training

#maybe we should check the sample to see if there are any tips in the dataset?

play with paramaters

In [ ]:
#increase iterations to 10

lrt = LogisticRegression(maxIter=10,
                        regParam=0.1,
                        elasticNetParam=0.3,
                        featuresCol="features",
                        labelCol="binarized_tip")

# Fit the model
lrModelt = lrt.fit(train_final)

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModelt.coefficients))
print("Intercept: " + str(lrModelt.intercept))

predictiont = lrModelt.transform(test)

pred_rddt= predictiont.select('prediction').rdd.flatMap(lambda x: x)
label_rddt = predictiont.select('binarized_tip').rdd.flatMap(lambda x: x)

predictionAndLabelst =  pred_rddt.zip(label_rddt)

metrics2t = MulticlassMetrics(predictionAndLabelst)

cmt = metrics2t.confusionMatrix().toArray()
acc = (cmt[0][0] + cmt[1][1])/(cmt[0][0] + cmt[1][1] + cmt[0][1] + cmt[1][0]) #should be a function

print(cmt)
print()
print(acc)

In [ ]:
#regParam to 0.01

lrt = LogisticRegression(maxIter=3,
                        regParam=0.01,
                        elasticNetParam=0.3,
                        featuresCol="features",
                        labelCol="binarized_tip")

# Fit the model
lrModelt = lrt.fit(train_final)

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModelt.coefficients))
print("Intercept: " + str(lrModelt.intercept))

predictiont = lrModelt.transform(test)

pred_rddt= predictiont.select('prediction').rdd.flatMap(lambda x: x)
label_rddt = predictiont.select('binarized_tip').rdd.flatMap(lambda x: x)

predictionAndLabelst =  pred_rddt.zip(label_rddt)

metrics2t = MulticlassMetrics(predictionAndLabelst)

cmt = metrics2t.confusionMatrix().toArray()
acc = (cmt[0][0] + cmt[1][1])/(cmt[0][0] + cmt[1][1] + cmt[0][1] + cmt[1][0]) #should be a function

print(cmt)
print()
print(acc)

In [ ]:
#elasticNet to 0

lrt = LogisticRegression(maxIter=3,
                        regParam=0.1,
                        elasticNetParam=0,
                        featuresCol="features",
                        labelCol="binarized_tip")

# Fit the model
lrModelt = lrt.fit(train_final)

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModelt.coefficients))
print("Intercept: " + str(lrModelt.intercept))

predictiont = lrModelt.transform(test)

pred_rddt= predictiont.select('prediction').rdd.flatMap(lambda x: x)
label_rddt = predictiont.select('binarized_tip').rdd.flatMap(lambda x: x)

predictionAndLabelst =  pred_rddt.zip(label_rddt)

metrics2t = MulticlassMetrics(predictionAndLabelst)

cmt = metrics2t.confusionMatrix().toArray()

acc = (cmt[0][0] + cmt[1][1])/(cmt[0][0] + cmt[1][1] + cmt[0][1] + cmt[1][0]) #should be a function

print(cmt)
print()
print(acc)

In [ ]:
#regParam to 0.9

lrt = LogisticRegression(maxIter=3,
                        regParam=0.9,
                        elasticNetParam=0.3,
                        featuresCol="features",
                        labelCol="binarized_tip")

# Fit the model
lrModelt = lrt.fit(train_final)

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModelt.coefficients))
print("Intercept: " + str(lrModelt.intercept))

predictiont = lrModelt.transform(test)

pred_rddt= predictiont.select('prediction').rdd.flatMap(lambda x: x)
label_rddt = predictiont.select('binarized_tip').rdd.flatMap(lambda x: x)

predictionAndLabelst =  pred_rddt.zip(label_rddt)

metrics2t = MulticlassMetrics(predictionAndLabelst)

cmt = metrics2t.confusionMatrix().toArray()

acc = (cmt[0][0] + cmt[1][1])/(cmt[0][0] + cmt[1][1] + cmt[0][1] + cmt[1][0]) #should be a function

print(cmt)
print()
print(acc)

In [ ]:
#regParam to 0.05 elasticNet = 0

lrt = LogisticRegression(maxIter=3,
                        regParam=0.05,
                        elasticNetParam=0,
                        featuresCol="features",
                        labelCol="binarized_tip")

# Fit the model
lrModelt = lrt.fit(train_final)

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModelt.coefficients))
print("Intercept: " + str(lrModelt.intercept))

predictiont = lrModelt.transform(test)

pred_rddt= predictiont.select('prediction').rdd.flatMap(lambda x: x)
label_rddt = predictiont.select('binarized_tip').rdd.flatMap(lambda x: x)

predictionAndLabelst =  pred_rddt.zip(label_rddt)

metrics2t = MulticlassMetrics(predictionAndLabelst)

cmt = metrics2t.confusionMatrix().toArray()

acc = (cmt[0][0] + cmt[1][1])/(cmt[0][0] + cmt[1][1] + cmt[0][1] + cmt[1][0]) #should be a function

print(cmt)
print()
print(acc)

In [ ]:
#regParam to 0.03 elasticNet = 0.1

lrt = LogisticRegression(maxIter=3,
                        regParam=0.03,
                        elasticNetParam=0.1,
                        featuresCol="features",
                        labelCol="binarized_tip")

# Fit the model
lrModelt = lrt.fit(train_final)

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModelt.coefficients))
print("Intercept: " + str(lrModelt.intercept))

predictiont = lrModelt.transform(test)

pred_rddt= predictiont.select('prediction').rdd.flatMap(lambda x: x)
label_rddt = predictiont.select('binarized_tip').rdd.flatMap(lambda x: x)

predictionAndLabelst =  pred_rddt.zip(label_rddt)

metrics2t = MulticlassMetrics(predictionAndLabelst)

cmt = metrics2t.confusionMatrix().toArray()

acc = (cmt[0][0] + cmt[1][1])/(cmt[0][0] + cmt[1][1] + cmt[0][1] + cmt[1][0]) #should be a function

print(cmt)
print()
print(acc)

In [ ]:
#regParam to 0.01 elasticNet = 0.1

lrt = LogisticRegression(maxIter=3,
                        regParam=0.01,
                        elasticNetParam=0.1,
                        featuresCol="features",
                        labelCol="binarized_tip")

# Fit the model
lrModelt = lrt.fit(train_final)

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModelt.coefficients))
print("Intercept: " + str(lrModelt.intercept))

predictiont = lrModelt.transform(test)

pred_rddt= predictiont.select('prediction').rdd.flatMap(lambda x: x)
label_rddt = predictiont.select('binarized_tip').rdd.flatMap(lambda x: x)

predictionAndLabelst =  pred_rddt.zip(label_rddt)

metrics2t = MulticlassMetrics(predictionAndLabelst)

cmt = metrics2t.confusionMatrix().toArray()

acc = (cmt[0][0] + cmt[1][1])/(cmt[0][0] + cmt[1][1] + cmt[0][1] + cmt[1][0]) #should be a function

print(cmt)
print()
print(acc)

In [ ]:
#regParam to 0.05 elasticNet = 0, maxIter = 10

lrt = LogisticRegression(maxIter=10,
                        regParam=0.05,
                        elasticNetParam=0,
                        featuresCol="features",
                        labelCol="binarized_tip")

# Fit the model
lrModelt = lrt.fit(train_final)

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModelt.coefficients))
print("Intercept: " + str(lrModelt.intercept))

predictiont = lrModelt.transform(test)

pred_rddt= predictiont.select('prediction').rdd.flatMap(lambda x: x)
label_rddt = predictiont.select('binarized_tip').rdd.flatMap(lambda x: x)

predictionAndLabelst =  pred_rddt.zip(label_rddt)

metrics2t = MulticlassMetrics(predictionAndLabelst)

cmt = metrics2t.confusionMatrix().toArray()

acc = (cmt[0][0] + cmt[1][1])/(cmt[0][0] + cmt[1][1] + cmt[0][1] + cmt[1][0]) #should be a function

print(cmt)
print()
print(acc)

In [ ]:
#regParam to 0.07 elasticNet = 0, maxIter = 10

lrt = LogisticRegression(maxIter=10,
                        regParam=0.07,
                        elasticNetParam=0,
                        featuresCol="features",
                        labelCol="binarized_tip")

# Fit the model
lrModelt = lrt.fit(train_final)

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModelt.coefficients))
print("Intercept: " + str(lrModelt.intercept))

predictiont = lrModelt.transform(test)

pred_rddt= predictiont.select('prediction').rdd.flatMap(lambda x: x)
label_rddt = predictiont.select('binarized_tip').rdd.flatMap(lambda x: x)

predictionAndLabelst =  pred_rddt.zip(label_rddt)

metrics2t = MulticlassMetrics(predictionAndLabelst)

cmt = metrics2t.confusionMatrix().toArray()

acc = (cmt[0][0] + cmt[1][1])/(cmt[0][0] + cmt[1][1] + cmt[0][1] + cmt[1][0]) #should be a function

print(cmt)
print()
print(acc)

In [ ]:
#regParam to 0.03 elasticNet = 0, maxIter = 10

lrt = LogisticRegression(maxIter=10,
                        regParam=0.03,
                        elasticNetParam=0,
                        featuresCol="features",
                        labelCol="binarized_tip")

# Fit the model
lrModelt = lrt.fit(train_final)

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModelt.coefficients))
print("Intercept: " + str(lrModelt.intercept))

predictiont = lrModelt.transform(test)

pred_rddt= predictiont.select('prediction').rdd.flatMap(lambda x: x)
label_rddt = predictiont.select('binarized_tip').rdd.flatMap(lambda x: x)

predictionAndLabelst =  pred_rddt.zip(label_rddt)

metrics2t = MulticlassMetrics(predictionAndLabelst)

cmt = metrics2t.confusionMatrix().toArray()

acc = (cmt[0][0] + cmt[1][1])/(cmt[0][0] + cmt[1][1] + cmt[0][1] + cmt[1][0]) #should be a function

print(cmt)
print()
print(acc)

In [ ]:
# enough wasting pixels
# can we make a lr pipeline using our notes from this week

In [ ]:
# from pyspark.ml import Pipeline  
# from pyspark.ml.feature import *  
# from pyspark.ml.classification import LogisticRegression

# # Configure pipeline stages
# # process review data
# tok = Tokenizer(inputCol="review", outputCol="words")  
# htf = HashingTF(inputCol="words", outputCol="tf", numFeatures=200)  

# # process review data
# w2v = Word2Vec(inputCol="review", outputCol="w2v")  

# # process rating data
# ohe = OneHotEncoder(inputCol="rating", outputCol="rc")  

# va = VectorAssembler(inputCols=["tf", "w2v", "rc"], outputCol="features")  
# lr = LogisticRegression(maxIter=10, regParam=0.01)

# # Build the pipeline
# pipeline = Pipeline(stages=[tok, htf, w2v, ohe, va, lr])

# # Fit the pipeline
# model = pipeline.fit(train_df)

# # Make a prediction
# prediction = model.transform(test_df)

In [ ]:
from pyspark.ml import Pipeline  
from pyspark.ml.feature import *  
from pyspark.ml.classification import LogisticRegression

# Configure pipeline stages

#onehotencoder to pickup
ohe_pu = OneHotEncoder(inputCol="Pickup_Community_Area", outputCol="Pickup_Community_Area_vec")
#df4 = onehotencoder_pickup_vector.transform(df4)

#onehotencoder to dropoff
ohe_do = OneHotEncoder(inputCol="Dropoff_Community_Area", outputCol="Dropoff_Community_Area_vec")
#df4 = onehotencoder_pickup_vector.transform(df4)

# our colulms for lr
predictor_col_for_lr = ['Trip_Seconds',
                        'Trip_Miles',
                        'Fare',
                        'Additional_Charges',
                        'Shared_Trip_Authorized',
                        'Trips_Pooled',
                        'Pickup_Community_Area_vec',
                        'Dropoff_Community_Area_vec']

lr_va = VectorAssembler(inputCols=predictor_col_for_lr, outputCol="features") 
# transformed = assembler.transform(df4)
# df5 = transformed

lr_scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        withStd=True, withMean=False)

#what do we want to do if we are doing a parameter search? make the parameters as variables and just do a loop?
lr = LogisticRegression(maxIter=10,
                        regParam=0.1,
                        elasticNetParam=0.3,
                        featuresCol="features",
                        labelCol="binarized_tip")

#split data
train_inital, test = df6.randomSplit([0.8, 0.2], seed=2021)

# Build the pipeline
data_pipeline = Pipeline(stages=[ohe_pu, ohe_do, lr_va, lr_scaler, lr])

# Fit the pipeline
lr_model = lr_pipeline.fit(train_final)

# Make a prediction
lr_prediction = lr_model.transform(test)

lr_pred_rdd= prediction.select('prediction').rdd.flatMap(lambda x: x)
lr_label_rddt = prediction.select('binarized_tip').rdd.flatMap(lambda x: x)

lr_predictionAndLabels =  lr_pred_rddt.zip(lr_label_rdd)

lr_metrics = MulticlassMetrics(lr_predictionAndLabels)

lr_cm = lr_metrics.confusionMatrix().toArray()

lr_acc = (lr_cm[0][0] + lr_cm[1][1])/(lr_cm[0][0] + lr_cm[1][1] + lr_cm[0][1] + lr_cm[1][0]) #should be a function

print(lr_cm)
print()
print(lr_acc)

In [ ]:
# # our model didn't work on the standard test train split.  Prof. Tashman recomended upscalling the help with the imbalanced dataset.
# #from https://spark.apache.org/docs/2.1.0/ml-tuning.html#train-validation-split

# train_inital, test = df6.randomSplit([0.8, 0.2], seed=2021)

# train_inital.show(5)

# test.cache()
# test.show(5)

In [ ]:
# # oversampleing code sample
# # https://stackoverflow.com/questions/53273133/how-to-perform-up-sampling-using-sample-functionpy-spark

# df_a = train_inital.filter(train_inital['binarized_tip'] == 0)
# df_b = train_inital.filter(train_inital['binarized_tip'] == 1)


In [ ]:
# a_count = df_a.count()
# b_count = df_b.count() 
# print(a_count)
# print(b_count)


In [ ]:
# ratio = a_count / b_count
# print(ratio)

In [ ]:
# df_b_overampled = df_b.sample(withReplacement=True, fraction=ratio, seed=1)

In [ ]:
# train_final = df_a.unionAll(df_b_overampled)
# train_final.cache()

In [ ]:
#used the in the lr.  This cell can be deleted

# https://medium.com/@nutanbhogendrasharma/role-of-onehotencoder-and-pipelines-in-pyspark-ml-feature-part-2-3275767e74f0
# apparently the spark doc is wrong, we don't need fit, just transform.  Maybe thats a spark 3 thing?

# from pyspark.ml.feature import OneHotEncoder

# #onehotencoder to pickup
# onehotencoder_pickup_vector = OneHotEncoder(inputCol="Pickup_Community_Area", outputCol="Pickup_Community_Area_vec")
# df4 = onehotencoder_pickup_vector.transform(df4)

# #onehotencoder to dropoff
# onehotencoder_dropoff_vector = OneHotEncoder(inputCol="Dropoff_Community_Area", outputCol="Dropoff_Community_Area_vec")
# df4 = onehotencoder_dropoff_vector.transform(df4)

# df4.show(20)c

In [37]:
# can we do RF in DF?
# from https://spark.apache.org/docs/latest/ml-classification-regression.html#random-forest-classifier

#I don't think we need this setup, we can use the same one from lr now

#lets start with the earlier datafram before any of the LR transforms
df8 =  df5.cache() 

# we need to regroup the features
# did this for lr.  skip

# RF doesn't like the community areas as they are too big.  Drop them for now
# predictor_col_for_rf = ['Trip_Seconds',
#                         'Trip_Miles',
#                         'Pickup_Community_Area_vec',
#                         'Dropoff_Community_Area_vec',
#                         'Fare',
#                         'Additional_Charges',
#                         'Shared_Trip_Authorized',
#                         'Trips_Pooled']


# from pyspark.ml.linalg import DenseVector
# from pyspark.ml.feature import VectorAssembler 
# from pyspark.mllib.linalg import Vectors

# assembler = VectorAssembler(inputCols=predictor_col_for_rf, outputCol="features") 
# transformed = assembler.transform(df8)
# df8 = transformed

df8.show(5)


+--------------------+--------------------+------------+----------+---------------------+----------------------+----+---+------------------+----------+----------------------+------------+-------------+-------------------------+--------------------------+--------------------+
|             Trip_ID|Trip_Start_Timestamp|Trip_Seconds|Trip_Miles|Pickup_Community_Area|Dropoff_Community_Area|Fare|Tip|Additional_Charges|Trip_Total|Shared_Trip_Authorized|Trips_Pooled|binarized_tip|Pickup_Community_Area_vec|Dropoff_Community_Area_vec|            features|
+--------------------+--------------------+------------+----------+---------------------+----------------------+----+---+------------------+----------+----------------------+------------+-------------+-------------------------+--------------------------+--------------------+
|1d7fc864aa5f775df...|12/01/2019 12:15:...|       543.0|       3.1|                  7.0|                   8.0| 7.5|0.0|              2.55|     10.05|                 fals

In [38]:
%whos

Variable                       Type               Data/Info
-----------------------------------------------------------
ArrayType                      type               <class 'pyspark.sql.types.ArrayType'>
Binarizer                      type               <class 'pyspark.ml.feature.Binarizer'>
BinaryType                     type               <class 'pyspark.sql.types.BinaryType'>
BooleanType                    type               <class 'pyspark.sql.types.BooleanType'>
ByteType                       type               <class 'pyspark.sql.types.ByteType'>
DataType                       type               <class 'pyspark.sql.types.DataType'>
DateType                       type               <class 'pyspark.sql.types.DateType'>
DecimalType                    type               <class 'pyspark.sql.types.DecimalType'>
DenseVector                    type               <class 'pyspark.ml.linalg.DenseVector'>
DoubleType                     type               <class 'pyspark.sql.types.DoubleT

In [39]:
del(df4)
del(df5)

In [40]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.

labelIndexer = StringIndexer(inputCol="binarized_tip", outputCol="indexedLabel").fit(df8)



In [41]:
type(labelIndexer)

pyspark.ml.feature.StringIndexerModel

In [42]:
# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.

# Set max to 78 as we have 77 CA plus out of city.  Doesn't work, max is 32.  This may move us to OHE.  Code can't handle CAs as it is.  Need to try something else. 
# short term to get code to run, drop pickup, droppoff areas.

# https://stackoverflow.com/questions/44959122/how-to-handle-categorical-features-for-decision-tree-random-forest-in-spark-ml
# https://spark.apache.org/docs/latest/ml-features.html#onehotencoder
# https://docs.databricks.com/_static/notebooks/binary-classification.html

featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=10).fit(df8)



In [43]:
type(featureIndexer)

pyspark.ml.feature.VectorIndexerModel

In [44]:
# Split the data into training and test sets (20% held out for testing)
# it won't be the same splits at this point, we'll have to fix that later once we have everything working

(trainingData_inital, testData) = df8.randomSplit([0.8, 0.2])

In [45]:
# oversampleing code sample
# https://stackoverflow.com/questions/53273133/how-to-perform-up-sampling-using-sample-functionpy-spark

df_a = trainingData_inital.filter(trainingData_inital['binarized_tip'] == 0)
df_b = trainingData_inital.filter(trainingData_inital['binarized_tip'] == 1)


In [46]:
a_count = df_a.count()
b_count = df_b.count() 
print(a_count)
print(b_count)


162981
32930


In [47]:
ratio = a_count / b_count
print(ratio)

4.9493167324628


In [48]:
df_b_overampled = df_b.sample(withReplacement=True, fraction=ratio, seed=1)

In [49]:
trainingData_final = df_a.unionAll(df_b_overampled).cache()

testData = testData.cache()

In [50]:
trainingData_final.show(5)

+--------------------+--------------------+------------+----------+---------------------+----------------------+----+---+------------------+----------+----------------------+------------+-------------+-------------------------+--------------------------+--------------------+
|             Trip_ID|Trip_Start_Timestamp|Trip_Seconds|Trip_Miles|Pickup_Community_Area|Dropoff_Community_Area|Fare|Tip|Additional_Charges|Trip_Total|Shared_Trip_Authorized|Trips_Pooled|binarized_tip|Pickup_Community_Area_vec|Dropoff_Community_Area_vec|            features|
+--------------------+--------------------+------------+----------+---------------------+----------------------+----+---+------------------+----------+----------------------+------------+-------------+-------------------------+--------------------------+--------------------+
|000e879212e519b17...|12/02/2019 01:45:...|      2376.0|      14.6|                 24.0|                  56.0|25.0|0.0|              7.55|     32.55|                 fals

In [51]:
df_af = trainingData_final.filter(trainingData_final['binarized_tip'] == 0)
df_bf = trainingData_final.filter(trainingData_final['binarized_tip'] == 1)
a_count = df_af.count()
b_count = df_bf.count() 
print(a_count)
print(b_count)


162981
163175


Our training size has increased.  This is to be expected in upscaling.

Good reference: https://www.kaggle.com/rafjaa/resampling-strategies-for-imbalanced-datasets

In [52]:
#just as a reminder what was the truth in our test data?

dft_a = testData.filter(testData['binarized_tip'] == 0)
dft_b = testData.filter(testData['binarized_tip'] == 1)
count_test_a = dft_a.count()
count_test_b = dft_b.count()
print(count_test_a)
print(count_test_b)


40583
8268


In [53]:
# Train a RandomForest model.
# these are all defalt settings
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

In [54]:
# Convert indexed labels back to original labels.
# more pipeline steps
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)


In [55]:
# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])


In [56]:
# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData_final)

In [57]:
# Make predictions.
predictions = model.transform(testData)

In [58]:
# Select example rows to display.
predictions.select("predictedLabel", "binarized_tip", "features").show(5)

+--------------+-------------+--------------------+
|predictedLabel|binarized_tip|            features|
+--------------+-------------+--------------------+
|           0.0|          0.0|(162,[0,1,2,3,5,3...|
|           0.0|          0.0|(162,[0,1,2,3,5,1...|
|           0.0|          0.0|(162,[0,1,2,4,5,2...|
|           0.0|          0.0|(162,[0,1,2,3,5,1...|
|           1.0|          0.0|(162,[0,1,2,3,5,3...|
+--------------+-------------+--------------------+
only showing top 5 rows



In [59]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

rfModel = model.stages[2]
print(rfModel)  # summary only

Test Error = 0.289308
RandomForestClassificationModel (uid=RandomForestClassifier_a094af962c62) with 10 trees


In [60]:
rf_pred_rdd= predictions.select('prediction').rdd.flatMap(lambda x: x)
rf_label_rdd = predictions.select('binarized_tip').rdd.flatMap(lambda x: x)

In [61]:
rf_predictionAndLabels =  rf_pred_rdd.zip(rf_label_rdd)

In [63]:
rf_metrics = MulticlassMetrics(rf_predictionAndLabels)
rf_cm = rf_metrics.confusionMatrix().toArray()
rf_acc = (rf_cm[0][0] + rf_cm[1][1])/(rf_cm[0][0] + rf_cm[1][1] + rf_cm[0][1] + rf_cm[1][0])

In [64]:
print(rf_cm)
print()
print(rf_acc)

[[31722.  8861.]
 [ 5272.  2996.]]

0.7106916951546539


In [65]:
#what is we play with our trees now 100

# Train a RandomForest model.
# these are all defalt settings
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=100)

# Convert indexed labels back to original labels.
# more pipeline steps
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData_final)

# Make predictions.
predictions = model.transform(testData)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

rfModel = model.stages[2]
print(rfModel)  # summary only

rf_pred_rdd= predictions.select('prediction').rdd.flatMap(lambda x: x)
rf_label_rdd = predictions.select('binarized_tip').rdd.flatMap(lambda x: x)
rf_predictionAndLabels =  rf_pred_rdd.zip(rf_label_rdd)

rf_metrics = MulticlassMetrics(rf_predictionAndLabels)
rf_cm = rf_metrics.confusionMatrix().toArray()
rf_acc = (rf_cm[0][0] + rf_cm[1][1])/(rf_cm[0][0] + rf_cm[1][1] + rf_cm[0][1] + rf_cm[1][0])

print(rf_cm)
print()
print(rf_acc)

Test Error = 0.28501
RandomForestClassificationModel (uid=RandomForestClassifier_8549690ea16a) with 100 trees
[[31911.  8672.]
 [ 5251.  3017.]]

0.7149904812593396


In [66]:
#what is we play with our trees now 10, max depth to 10

# Train a RandomForest model.
# these are all defalt settings
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10, maxDepth=10)

# Convert indexed labels back to original labels.
# more pipeline steps
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData_final)

# Make predictions.
predictions = model.transform(testData)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

rfModel = model.stages[2]
print(rfModel)  # summary only

rf_pred_rdd= predictions.select('prediction').rdd.flatMap(lambda x: x)
rf_label_rdd = predictions.select('binarized_tip').rdd.flatMap(lambda x: x)
rf_predictionAndLabels =  rf_pred_rdd.zip(rf_label_rdd)

rf_metrics = MulticlassMetrics(rf_predictionAndLabels)
rf_cm = rf_metrics.confusionMatrix().toArray()
rf_acc = (rf_cm[0][0] + rf_cm[1][1])/(rf_cm[0][0] + rf_cm[1][1] + rf_cm[0][1] + rf_cm[1][0])

print(rf_cm)
print()
print(rf_acc)

Test Error = 0.363084
RandomForestClassificationModel (uid=RandomForestClassifier_57c53127580d) with 10 trees
[[26758. 13825.]
 [ 3912.  4356.]]

0.6369163374342388


In [67]:
#what is we play with our trees now 10, max depth to 20

# Train a RandomForest model.
# these are all defalt settings
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10, maxDepth=20)

# Convert indexed labels back to original labels.
# more pipeline steps
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData_final)

# Make predictions.
predictions = model.transform(testData)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

rfModel = model.stages[2]
print(rfModel)  # summary only

rf_pred_rdd= predictions.select('prediction').rdd.flatMap(lambda x: x)
rf_label_rdd = predictions.select('binarized_tip').rdd.flatMap(lambda x: x)
rf_predictionAndLabels =  rf_pred_rdd.zip(rf_label_rdd)

rf_metrics = MulticlassMetrics(rf_predictionAndLabels)
rf_cm = rf_metrics.confusionMatrix().toArray()
rf_acc = (rf_cm[0][0] + rf_cm[1][1])/(rf_cm[0][0] + rf_cm[1][1] + rf_cm[0][1] + rf_cm[1][0])

print(rf_cm)
print()
print(rf_acc)

Test Error = 0.386809
RandomForestClassificationModel (uid=RandomForestClassifier_1c374aa63e6c) with 10 trees
[[25118. 15465.]
 [ 3431.  4837.]]

0.6131911322183783


deeper trees seems to be worse, and slower

In [68]:
#what is we play with our trees now 1000

# Train a RandomForest model.
# these are all defalt settings
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=1000)

# Convert indexed labels back to original labels.
# more pipeline steps
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData_final)

# Make predictions.
predictions = model.transform(testData)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

rfModel = model.stages[2]
print(rfModel)  # summary only

rf_pred_rdd= predictions.select('prediction').rdd.flatMap(lambda x: x)
rf_label_rdd = predictions.select('binarized_tip').rdd.flatMap(lambda x: x)
rf_predictionAndLabels =  rf_pred_rdd.zip(rf_label_rdd)

rf_metrics = MulticlassMetrics(rf_predictionAndLabels)
rf_cm = rf_metrics.confusionMatrix().toArray()
rf_acc = (rf_cm[0][0] + rf_cm[1][1])/(rf_cm[0][0] + rf_cm[1][1] + rf_cm[0][1] + rf_cm[1][0])

print(rf_cm)
print()
print(rf_acc)

Test Error = 0.309635
RandomForestClassificationModel (uid=RandomForestClassifier_eec0ebddb69e) with 1000 trees
[[30268. 10315.]
 [ 4811.  3457.]]

0.6903645780024974


In [69]:
#what is we play with our trees now 500

# Train a RandomForest model.
# these are all defalt settings
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=500)

# Convert indexed labels back to original labels.
# more pipeline steps
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData_final)

# Make predictions.
predictions = model.transform(testData)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

rfModel = model.stages[2]
print(rfModel)  # summary only

rf_pred_rdd= predictions.select('prediction').rdd.flatMap(lambda x: x)
rf_label_rdd = predictions.select('binarized_tip').rdd.flatMap(lambda x: x)
rf_predictionAndLabels =  rf_pred_rdd.zip(rf_label_rdd)

rf_metrics = MulticlassMetrics(rf_predictionAndLabels)
rf_cm = rf_metrics.confusionMatrix().toArray()
rf_acc = (rf_cm[0][0] + rf_cm[1][1])/(rf_cm[0][0] + rf_cm[1][1] + rf_cm[0][1] + rf_cm[1][0])

print(rf_cm)
print()
print(rf_acc)

Test Error = 0.324599
RandomForestClassificationModel (uid=RandomForestClassifier_811859f57f5a) with 500 trees
[[29337. 11246.]
 [ 4611.  3657.]]

0.6754007082761868


In [70]:
#what is we play with our trees now 50

# Train a RandomForest model.
# these are all defalt settings
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=50)

# Convert indexed labels back to original labels.
# more pipeline steps
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData_final)

# Make predictions.
predictions = model.transform(testData)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

rfModel = model.stages[2]
print(rfModel)  # summary only

rf_pred_rdd= predictions.select('prediction').rdd.flatMap(lambda x: x)
rf_label_rdd = predictions.select('binarized_tip').rdd.flatMap(lambda x: x)
rf_predictionAndLabels =  rf_pred_rdd.zip(rf_label_rdd)

rf_metrics = MulticlassMetrics(rf_predictionAndLabels)
rf_cm = rf_metrics.confusionMatrix().toArray()
rf_acc = (rf_cm[0][0] + rf_cm[1][1])/(rf_cm[0][0] + rf_cm[1][1] + rf_cm[0][1] + rf_cm[1][0])

print(rf_cm)
print()
print(rf_acc)

Test Error = 0.281939
RandomForestClassificationModel (uid=RandomForestClassifier_f1194e034f89) with 50 trees
[[32152.  8431.]
 [ 5342.  2926.]]

0.7180610427626866


In [74]:
#try a GBT 

# https://spark.apache.org/docs/latest/ml-classification-regression.html#gradient-boosted-tree-classifier

In [71]:

from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Load and parse the data file, converting it to a DataFrame.
# data = spark.read.format("libsvm").load("data/mllib/sample_libsvm_data.txt")
# use same data as rf


# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
#labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)
# use same data as rf

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
# featureIndexer =\
#     VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
# (trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a GBT model.
gbt = GBTClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", maxIter=10)

# Chain indexers and GBT in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, gbt])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData_final)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "indexedLabel", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

gbtModel = model.stages[2]
print(gbtModel)  # summary only

gbt_pred_rdd= predictions.select('prediction').rdd.flatMap(lambda x: x)
gbt_label_rdd = predictions.select('binarized_tip').rdd.flatMap(lambda x: x)
gbt_predictionAndLabels =  gbt_pred_rdd.zip(gbt_label_rdd)

gbt_metrics = MulticlassMetrics(gbt_predictionAndLabels)
gbt_cm = gbt_metrics.confusionMatrix().toArray()
gbt_acc = (gbt_cm[0][0] + gbt_cm[1][1])/(gbt_cm[0][0] + gbt_cm[1][1] + gbt_cm[0][1] + gbt_cm[1][0])

print(gbt_cm)
print()
print(gbt_acc)


+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       0.0|         0.0|(162,[0,1,2,3,5,3...|
|       1.0|         0.0|(162,[0,1,2,3,5,1...|
|       0.0|         0.0|(162,[0,1,2,4,5,2...|
|       1.0|         0.0|(162,[0,1,2,3,5,1...|
|       1.0|         0.0|(162,[0,1,2,3,5,3...|
+----------+------------+--------------------+
only showing top 5 rows

Test Error = 0.372091
GBTClassificationModel (uid=GBTClassifier_86f44db64d7b) with 10 trees
[[26294. 14289.]
 [ 3888.  4380.]]

0.6279093570244212


In [72]:
# Train a GBT model.
gbt = GBTClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", maxIter=50)

# Chain indexers and GBT in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, gbt])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData_final)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "indexedLabel", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

gbtModel = model.stages[2]
print(gbtModel)  # summary only

gbt_pred_rdd= predictions.select('prediction').rdd.flatMap(lambda x: x)
gbt_label_rdd = predictions.select('binarized_tip').rdd.flatMap(lambda x: x)
gbt_predictionAndLabels =  gbt_pred_rdd.zip(gbt_label_rdd)

gbt_metrics = MulticlassMetrics(gbt_predictionAndLabels)
gbt_cm = gbt_metrics.confusionMatrix().toArray()
gbt_acc = (gbt_cm[0][0] + gbt_cm[1][1])/(gbt_cm[0][0] + gbt_cm[1][1] + gbt_cm[0][1] + gbt_cm[1][0])

print(gbt_cm)
print()
print(gbt_acc)

+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       0.0|         0.0|(162,[0,1,2,3,5,3...|
|       1.0|         0.0|(162,[0,1,2,3,5,1...|
|       0.0|         0.0|(162,[0,1,2,4,5,2...|
|       1.0|         0.0|(162,[0,1,2,3,5,1...|
|       1.0|         0.0|(162,[0,1,2,3,5,3...|
+----------+------------+--------------------+
only showing top 5 rows

Test Error = 0.408262
GBTClassificationModel (uid=GBTClassifier_a46ec092a825) with 50 trees
[[23692. 16891.]
 [ 3053.  5215.]]

0.5917381425149946


In [73]:
# Train a GBT model.
gbt = GBTClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", maxIter=10, maxDepth=10)

# Chain indexers and GBT in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, gbt])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData_final)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "indexedLabel", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

gbtModel = model.stages[2]
print(gbtModel)  # summary only

gbt_pred_rdd= predictions.select('prediction').rdd.flatMap(lambda x: x)
gbt_label_rdd = predictions.select('binarized_tip').rdd.flatMap(lambda x: x)
gbt_predictionAndLabels =  gbt_pred_rdd.zip(gbt_label_rdd)

gbt_metrics = MulticlassMetrics(gbt_predictionAndLabels)
gbt_cm = gbt_metrics.confusionMatrix().toArray()
gbt_acc = (gbt_cm[0][0] + gbt_cm[1][1])/(gbt_cm[0][0] + gbt_cm[1][1] + gbt_cm[0][1] + gbt_cm[1][0])

print(gbt_cm)
print()
print(gbt_acc)

+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       0.0|         0.0|(162,[0,1,2,3,5,3...|
|       1.0|         0.0|(162,[0,1,2,3,5,1...|
|       0.0|         0.0|(162,[0,1,2,4,5,2...|
|       1.0|         0.0|(162,[0,1,2,3,5,1...|
|       1.0|         0.0|(162,[0,1,2,3,5,3...|
+----------+------------+--------------------+
only showing top 5 rows

Test Error = 0.376267
GBTClassificationModel (uid=GBTClassifier_e2dd117300c1) with 10 trees
[[25879. 14704.]
 [ 3677.  4591.]]

0.6237333933798694


stuff coppied for other Tahsman Notebooks that might be useful.  Not developed yet.

In [ ]:
# for each field, compute missing percentage
# from preprcessing example notebook

df.agg(*[
    (1 - F.count(c) / F.count('*')).alias(c + '_miss')
    for c in df.columns
]).show()

In [ ]:
#bayes example

In [ ]:
from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel
from pyspark.mllib.util import MLUtils

# Load the data file. Note this data is in sparse format.
data = MLUtils.loadLibSVMFile(sc, 'sample_libsvm_data.txt')
data.take(2)

In [ ]:
# Split data approximately into training (60%) and test (40%)
training, test = data.randomSplit([0.6, 0.4])

In [ ]:
# Train a naive Bayes model.
model = NaiveBayes.train(training, 1.0)

In [ ]:
# Make prediction and test accuracy.
labelsAndPreds = test.map(lambda p: (p.label, model.predict(p.features)))
accuracy = 1.0 * labelsAndPreds.filter(lambda pl: pl[0] == pl[1]).count() / test.count()
print('model accuracy {}'.format(accuracy))

# Source: https://spark.apache.org/docs/latest/mllib-naive-bayes.html

In [ ]:
#decision tree examples

In [ ]:
from pyspark.mllib.tree import DecisionTree
from pyspark.mllib.util import MLUtils

# Load and parse the data file
data = MLUtils.loadLibSVMFile(sc, 'sample_libsvm_data.txt')
data.take(2)

In [ ]:
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [ ]:
# Train a DecisionTree model.
#  Empty categoricalFeaturesInfo indicates all features are continuous.
model = DecisionTree.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                     impurity='gini', maxDepth=5, maxBins=32)

In [ ]:
# Evaluate model on test instances and compute test error
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(
    lambda lp: lp[0] != lp[1]).count() / float(testData.count())
print('Test Error = ' + str(testErr))

In [ ]:
# my personal favorite... trees

**Tree-Based Ensemble Methods**

*Ensembles* combine multiple models together to produce a new model.  
They may consist of models of the same type (e.g., all decision trees) or mixed type (e.g., decision tree + neural net + svm)  

One of the fundamental results in machine learning is that multiple weak classifiers can be combined to produce a strong classifier.  

Ensembles are useful in reducing overfitting, since predictions are based on several different trees  

The two most popular tree-based ensemble methods are *Random Forests* and *Boosted Trees* (e.g. *Gradient-Boosted Trees*)  

They are popular because they are often very competitive  

The nice properties of decision trees carry over to ensembles of trees  

This combining step can proceed using different methods, including:  

- voting (for classification)
- averaging (for regression) 
- running model predictions through another model (classification and regression)

There are downsides to ensembles:  

- Multiple models need to be trained, loaded, and maintained  
- Model explanation is harder: no p-values like regression, several trees are feeding overall decision.  
There are methods to provide feature importance information, such as partial dependence plots.

**Random Forest**  
Ensembles of decision trees  

RFs inject two sources of randomness into modeling:  

1. At each step, randomly select $p$ features out of $n$ total features for possible inclusion (random subspace method)
2. Sample the original training set with replacement, up to the size of the original training set (bootstrapping of the training set)

The number of features to randomly select $p$ is a parameter  
The number of bootstrapped trees to grow $N$ is a parameter  

Since the trees are grown independently, the training and prediction tasks are embarrassingly parallel and can be assigned to multiple workers.

Classification prediction done by majority vote across trees

**Random Forest Implementation**

`from pyspark.mllib.tree import RandomForest`  

Two most important parameters (which should be tuned using $k$-fold cross validation):  

- `numTrees`: Number of trees in forest
More trees will increase accuracy but also training time  

- `maxDepth`: Maximum depth of each tree in forest
Increasing depth can increase power of model, but will take longer to train and can overfit  

Other important parameters:

- `subsamplingRate`: fraction of size of original training set (default=1.0 recommended)

- `featureSubsetStrategy`: specified as fraction or function of total number of features

**Random Forest Example: load data/train model/predict**  
NOTE: Very similar to Decision Tree code above


In [ ]:
from pyspark.mllib.tree import RandomForest
from pyspark.mllib.util import MLUtils

data = MLUtils.loadLibSVMFile(sc, 'sample_libsvm_data.txt')
data.take(2)

In [ ]:
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [ ]:
# Train a RandomForest model.
#  Empty categoricalFeaturesInfo indicates all features are continuous.
#  Setting featureSubsetStrategy="auto" lets the algorithm choose.
model = RandomForest.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                     numTrees=1000, featureSubsetStrategy="auto",
                                     impurity='gini', maxDepth=5, maxBins=32)

In [ ]:
# Evaluate model on test instances and compute test error
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(
    lambda lp: lp[0] != lp[1]).count() / float(testData.count())
print('Test Error = ' + str(testErr))

**Gradient-Boosted Trees**  

GBTs work by building a sequence of trees and combining their predictions at each iteration.  The trees constructed are generally *stumps* which use a single decision split.  A stump is an example of a weak learner.

This is different from random forests, where each tree independently gives predictions on each training instance.



A loss is specified and an optimization problem is solved whereby the objective is to minimize the loss of the model by adding weak learners using a gradient-descent-like procedure.

The procedure follows a stage-wise additive model, meaning that one new weak learner is
added at a time and existing weak learners are left unchanged.
For the original work, see:

*Friedman, Jerome H. "Greedy function approximation: a gradient boosting machine." Annals of Statistics (2001): 1189–1232.*


**Gradient-Boosted Trees Implementation**  

Since the trees are built in a sequential fashion, the algorithm can not be run in parallel.  
However, shallow trees (e.g., stumps) can be used effectively; this saves time versus random forests, which use deeper trees.

The loss function in classification problems is the log loss, equal to twice the binomial negative log likelihood.

Important parameters:
- `numIterations`:  equal to the number of trees in the ensemble.  More trees means longer runtime but also better performance up to a point.
- `learningRate`:  how quickly the model adapts on each iteration. A smaller value may help the algo have better performance, but at the cost of additional runtime. The documentation recommends NOT tuning this param.

The method `runWithValidation` can help mitigate overfitting.  It takes a training RDD and a validation RDD.

The training is stopped when the improvement in the validation error is not more than a certain tolerance (supplied by the `validationTol` argument in `BoostingStrategy`).

**GBT Example: load data/train model/predict**